## **Sistemas de Recomendação <br>**
<br>

Sistemas de recomendação desempenham papel importante na personalização da experiência do usuário em vários tipos de plataformas e serviços online, já que existe a disponibilidade de um número quase ilimitado de opções (por exemplo, diferentes gêneros de filmes de qualidade variada), o usuário precisa de orientação para o próximo item que atenda às suas expectativas.

### **Tipos de sistemas <br>**
<br>
Existem muitas maneiras e algoritmos usados ​​para construir um sistema de recomendação. As principais abordagens são: Item Mais Popular, Associação/Modelo de Cesta, Filtragem de Conteúdo, Filtragem Colaborativa (baseado no usuário e/ou no item) e Modelos Híbridos.

Intuitivamente, os Modelos Híbridos parecem ser mais efetivos, já que combinam duas ou mais estratégias de recomendação, porém, neste trabalho, vamos assumir que usuários semelhantes avaliam de forma semelhante os filmes e usar os métodos de filtragem colaborativa, mais especificamente o SVD e NMF, considerando a abordagem baseada nos itens (filmes).

### **User-based vs Item-Based <br>**

Lembrando que a abordagem deste trabalho é supondo que usuários semelhantes exibem preferências semelhantes. Para determinar o grupo de referência no modelo **user-based**, leva-se em consideração que tanto o usuário quanto o grupo compartilham um histórico de classificação de itens semelhantes. Na próxima etapa, o sistema aloca os únicos itens aos quais o grupo de referência foi exposto anteriormente e recomenda os itens do usuário final com base na classificação dos usuários do grupo que avaliaram aquele item. As técnicas colaborativas **item-based** analisam a matriz item-usuário e identificam relações entre diferentes itens. O sistema de recomendação baseado em itens faz, então, recomendações com base nas relações lineares (semelhanças) entre os itens. <br> <br>


Este trabalho avalia a Filtragem Colaborativa Baseada em Itens. No contexto de uma recomendação de filme para filme, um filtro colaborativo responde à pergunta: “*Quais filmes têm um perfil de avaliação de usuário semelhante*?”.

# Leitura dos dados

Definindo variável flag que busca dados de fontes diferentes (AWS S3 ou Group Lens):

In [17]:
ENV_AWS = False

## **Conjunto de dados analisado <br>**
<br>

MovieLens é um site de recomendação de filmes, administrado pela GroupLens Research da Universidade de Minnesota, que usa tecnologia de filtragem colaborativa para fazer recomendações de filmes. A GroupLens Research coletou e disponibilizou conjuntos de dados de classificação do site MovieLens (https://movielens.org), para pesquisadores que apresentem interesse em evoluir no aprendizado de personalização e tecnologias de filtragem.

No site https://grouplens.org/datasets/movielens/ há vários conjuntos de dados disponibilizados, escolhemos estudar o conjunto que contém 25 milhões de avaliações, que foram feitas em 62 mil filmes por 162 mil usuários. Os dados foram gerados entre janeiro de 1995 e novembro de 2019. <br>

Os usuários foram selecionados aleatoriamente para inclusão. Todos os usuários selecionados avaliaram pelo menos 20 filmes. Não há informações demográficas disponíveis. Cada usuário é representado por um ID e nenhuma outra informação do usuário é fornecida.

Os dados estão divididos em  alguns arquivos e neste trabalho foram usados os seguintes arquivos:

  * movies.csv: ID do filme, título do filme e gênero do filme (aventura, animação, criança etc) - usado na análise descritiva;
  * ratings.csv: ID do usuário, ID do filme, nota dada pelo usuário ao filme e data da avaliação - usado nos modelos de machine learning. O usuário avalia o filme com uma nota entre 0.5 e 5, sendo que quanto maior a nota, mais ele gostou do filme.;
  * tags.csv: ID do usuário, ID do filme, tag dada pelo usuário ao filme (clássico, ficção científica, comédia etc) e data da avaliação - usado na análise descritiva.


In [18]:
import requests
import zipfile
import pandas as pd
import numpy as np
import os
import sys
import matplotlib.pyplot as plt
import seaborn as sns
import boto3

In [19]:
if not ENV_AWS:
    filename = "ml-25m.zip"
    url = f"https://files.grouplens.org/datasets/movielens/{filename}"
    query_parameters = {"downloadformat": "zip"}

    response = requests.get(url, params=query_parameters)

    with open(filename, mode="wb") as file:
        file.write(response.content)

In [20]:
if not ENV_AWS:
    with zipfile.ZipFile(filename,"r") as zip_ref:
        zip_ref.extractall()

In [21]:
if not ENV_AWS:
    folder = "ml-25m"
    df_ratings = pd.read_csv(f"{folder}/ratings.csv")
    df_ratings.head()

Selecionando apenas as colunas que serão usadas.

In [22]:
if not ENV_AWS:
    notas=df_ratings[['userId',  'movieId', 'rating']]
    notas.head()
    print(notas.shape)

No banco de dados existem apenas usuários que avaliaram pelo menos 20 filmes, porém filmes com poucas avaliações também podem aumentar a instabilidade das predições, por isso manteremos no banco de dados apenas filmes com pelo menos 20 avaliações, removendo, então 0,7% das avaliações.

In [23]:
if not ENV_AWS:
    notas = notas.groupby("movieId").filter(lambda x: x['movieId'].count() >= 20)
    print(notas.shape)

<font color=red> Amostragem para rodar no colab.</font>

In [24]:
if not ENV_AWS:
    notas = notas.sample(n=100000, replace=False, random_state=1234)

Lendo os dados para quando já temos os arquivos pré-processados:

In [25]:
if ENV_AWS:
    s3 = boto3.client("s3")

    s3.download_file(
        Bucket="cdd-ratings",
        Key="ratings.parquet",
        Filename="ratings.parquet",
    )
    notas = pd.read_parquet("ratings.parquet")

Instalação da biblioteca Surprise e leitura das funções que serão usadas.

In [29]:
!pip install surprise

In [30]:
from surprise import Reader, dataset, NMF, KNNWithMeans, SVD, accuracy
from surprise.model_selection import train_test_split, GridSearchCV, cross_validate

Transformação do pandas dataset em surprise (sparce matrix) dataset.

In [31]:
reader = Reader(line_format='user item rating', rating_scale=(0.5, 5))

In [32]:
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userId'], df['movieId'], df['rating'])]
        self.reader=reader

data = MyDataset(notas, reader)

Divisão do banco em treino e teste.

In [33]:
trainset, testset = train_test_split(data, test_size=.20, random_state=1234)

## **Treinamento dos modelos** <br> <br>

### **Centered KNN** <br>

O KNN é um algoritmo baseado em memória, ou seja, é aplicado em todo o banco de dados para calcular as predições. Na biblioteca surprise, o Centered KNN é nomeado como KNNWithMeans e aceita 3 hiperparâmetros k (máximo número de vizinhos), min_k (número mínimo de vizinhos) e sim_options (dicionário de opções sobre as medidas de similaridade).Esse algoritmo leva em consideração a nota média de cada usuário. <br>

Nesta análise usamos o cosseno como medida de similaridade, modelo baseado em itens e o default do k (40) e min_k(20)

In [34]:
sim_options = {
    "name": "cosine", # usa o cosseno como medida de similaridade
    "user_based": False,  # calcula similaridade entre itens
}
algoritmoknn = KNNWithMeans(sim_options=sim_options)
algoritmoknn.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.


### Acurácia <br>

A medida de acurácia escolhida nesta análise é o RMSE, que calcula a raiz dos erros quadráticos médio, ou seja, para cada observação, é calculada a diferença entre o valor real e o predito e essa diferença é elevada ao quadrado. Então é obtida a média de todas as diferenças e por fim é extraída a raiz quadrada. Com isso, é intuítivo, que o RMSE penaliza mais diferenças maiores. Para a interpretação, devemos considerar que as notas de avaliação variam entre 0,5 e 5, portanto, um RMSE de 1 corresponde 22% da escala e já parece ser razoavelmente alto.

In [35]:
test_train = trainset.build_testset()
predictions = algoritmoknn.test(test_train)
accuracy.rmse(predictions)

RMSE: 0.5004


0.5003686853164521

Ao interpretar o RMSE calculado no banco de treinamento, devemos lembrar que essa medida é viesada, já que está calculada no mesmo conjunto de dados em que ocorreu o treinamento, ou seja, a medida tende a ser menor. Porém é um bom ponto de partida para escolher o modelo que sofrerá tunagem de hiperparâmetros.

As predições, no pacote surprise, retornam o id do usuário (uid), o id do filme (iid), a avaliação real do usuário (r_ui), o valor predito pelo algoritmo (est), os detalhes do algoritmo e a sentença 'was_impossible' que retorna True quando o valor predito (est) não está na escala de variação das avaliações.

In [36]:
predictions[0:3]

[Prediction(uid=37945, iid=2568, r_ui=0.5, est=1.212877645637255, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=37945, iid=3617, r_ui=4.0, est=2.750233967476335, details={'actual_k': 8, 'was_impossible': False}),
 Prediction(uid=37945, iid=30810, r_ui=3.5, est=3.1522715850311944, details={'actual_k': 8, 'was_impossible': False})]

### **SVD** <br>

SVD (Singular Value Decomposition) é um algoritmo de fatoração de matrizes, cujo objetivo é reduzir a dimensionalidade para diminuir o custo computacional, já que se beneficia das propriedades da álgebra linear. Em resumo, a fatoração de matrizes pode ser vista como a decomposição de uma matriz grande em um produto de matrizes menores. Isso é semelhante à fatoração de números inteiros, onde 12 pode ser escrito como 6x2 ou 4x3, por exemplo. No caso de matrizes, uma matriz A com dimensões m x n pode ser reduzida a um produto de duas matrizes X e Y com dimensões m x p e p x n respectivamente. <br><br>

SVD decompõe qualquer matriz em vetores singulares e valores singulares. O objetivo geral do SVD é decompor a matriz R com todos os elementos faltantes e, posteriormente, multiplicar seus componentes. Como resultado, não há valores ausentes e é possível recomendar a cada usuário filmes (itens) que ainda não viram. <br><br>

Na biblioteca surprise, o SVD é nomeado como SVD e aceita 16 hiperparâmetros, entre eles o número de fatores (n_factors), o número de iterações (n_epochs), alguns parâmetros de aprendizagem e de regularização.

Nesta análise mantemos todos os hiperparâmetros default, fixando apenas o random_state.

In [37]:
algoritmoSVD = SVD(random_state=1234)
algoritmoSVD.fit(trainset)

In [38]:
predictions = algoritmoSVD.test(test_train)
accuracy.rmse(predictions)

RMSE: 0.6836


0.6835602245943805

In [39]:
predictions[0:3]

[Prediction(uid=37945, iid=2568, r_ui=0.5, est=2.0930145048981434, details={'was_impossible': False}),
 Prediction(uid=37945, iid=3617, r_ui=4.0, est=3.0563772097368003, details={'was_impossible': False}),
 Prediction(uid=37945, iid=30810, r_ui=3.5, est=3.1073394761870055, details={'was_impossible': False})]

### **NMF** <br>

NMF (Non-negative Matrix Factorization) também é um algoritmo de fatoração de matrizes. O NMF decompõe a matriz não negativa em duas outras matrizes, em que as colunas da primeira representam os componentes e a segunda matriz armazena os pesos. O NMF coloca restrições para que as duas matrizes formadas sejam não negativas. Essa restrição é a principal diferença em relação ao SVD. <br><br>

Na biblioteca surprise, o NMF é nomeado como NMF e aceita 12 hiperparâmetros, entre eles o número de fatores (n_factors), o número de iterações (n_epochs), alguns parâmetros de aprendizagem e de regularização.

Nesta análise mantemos todos os hiperparâmetros default, fixando apenas o random_state.

In [40]:
algoritmoNMF = NMF(random_state=1234)
algoritmoNMF.fit(trainset)

In [41]:
predictions = algoritmoNMF.test(test_train)
accuracy.rmse(predictions)

RMSE: 0.1085


0.1085418049233949

Avaliando o RMSE no conjunto de treinamento, nota-se que o modelo com melhor perfomance é o NMF (RMSE=0,11). Portanto, vamos realizar o refinamento de hiperparâmetros neste modelo.

In [42]:
param_grid = {'n_factors': [10,15,20], #Default é 15
              'n_epochs': [30,50,70], #Default é 50
              'reg_qi':[0.01,0.06,0.1], #Default é 0.06
              'random_state':[1234]}
grid_search = GridSearchCV(NMF, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)

In [43]:
print(grid_search.best_score['rmse'])

1.1171703024296569


Note que o RMSE da melhor combinação de hiperparâmetros ficou muito maior que o RMSE calculado no conjunto de treinamento, indicando que houve sobreajuste no treinamento. Portanto, vamos realizar o refinamento de hiperparâmetros no SVD, para comparar os resultados.

In [44]:
from surprise.model_selection import GridSearchCV

param_grid = {'n_factors': [100,150], # defualt=100
              'n_epochs': [20,25,30], # defualt=20
              'lr_all':[0.005,0.01,0.1], # defualt=0.005
              'reg_all':[0.02,0.05,0.1], # defualt=0.02
              'random_state':[1234]}
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse','mae'], cv=3)
grid_search.fit(data)

In [45]:
print(grid_search.best_score['rmse'])

0.9695968583102935


Com o RMSE mais baixo que o NMF, escolhemos aplicar validação cruzada no algorotmo SVD, com os melhores hiperparâmetros.

In [46]:
print(grid_search.best_params['rmse'])

{'n_factors': 100, 'n_epochs': 30, 'lr_all': 0.01, 'reg_all': 0.1, 'random_state': 1234}


In [47]:
algo = grid_search.best_estimator['rmse']

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9677  0.9670  0.9602  0.9625  0.9665  0.9648  0.0029  
MAE (testset)     0.7514  0.7463  0.7477  0.7464  0.7517  0.7487  0.0024  
Fit time          2.15    2.21    2.30    2.24    2.23    2.23    0.05    
Test time         0.12    0.12    0.41    0.12    0.12    0.18    0.12    


{'test_rmse': array([0.96765428, 0.96699677, 0.96021291, 0.96246056, 0.96653025]),
 'test_mae': array([0.75136803, 0.74629508, 0.74768479, 0.7463834 , 0.75174061]),
 'fit_time': (2.153818130493164,
  2.2105705738067627,
  2.3010435104370117,
  2.240523099899292,
  2.2341833114624023),
 'test_time': (0.11640453338623047,
  0.1183478832244873,
  0.4080648422241211,
  0.1183159351348877,
  0.1156923770904541)}

O RMSE médio observado foi de 0,96, representando 21% da variação da escala, ou seja, obtemos uma performance média. Dado o desvio padrão baixo (0,004), nota-se que o algoritmo é bem comportado neste banco de dados e pode ser usado para sugerir filmes para usuários.

Não foi possível fazer refinamento de parâmetros no KNN, que apresentou a segunda melhor acurácia no banco de treinamento, porém avaliando a acurácia com validação cruzada, observou-se que esse algoritmo apresetou maior RMSE que o SVD.

In [48]:
sim_options = {
    "name": "cosine", # usa o cosseno como medida de similaridade
    "user_based": False,  # calcula similaridade entre itens
}
algoritmoknn = KNNWithMeans(sim_options=sim_options)

cross_validate(algoritmoknn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0396  1.0399  1.0392  1.0349  1.0343  1.0375  0.0025  
MAE (testset)     0.8158  0.8135  0.8150  0.8125  0.8121  0.8138  0.0014  
Fit time          1.86    1.91    1.95    1.92    2.01    1.93    0.05    
Test time         0.18    0.17    0.50    0.18    0.18    0.24    0.13    


{'test_rmse': array([1.03955476, 1.03989114, 1.03915688, 1.03486164, 1.03426199]),
 'test_mae': array([0.81581581, 0.81348499, 0.81498096, 0.8125062 , 0.81206897]),
 'fit_time': (1.8598248958587646,
  1.907480001449585,
  1.9454433917999268,
  1.9173204898834229,
  2.011765956878662),
 'test_time': (0.17692208290100098,
  0.1704695224761963,
  0.5004315376281738,
  0.1767103672027588,
  0.18084478378295898)}

## **Trabalhos Futuros** <br>

Avaliar se existe oportunidade de melhorar a performance da recomendação testando outros algorítmos, até mesmo usando modelos híbridos a partir de deep learning.